In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
from backend.extractor.utils import parse_html, save_html
from backend.extractor.utils import clean_html, prepare_html, traverse_and_modify
from backend.fetcher.fetcher import fetch_page
import asyncio
from bs4 import BeautifulSoup

In [155]:
import re


def load_html_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        html = file.read()
    return html

def clean_html(html):
    soup = BeautifulSoup(html, "html.parser")

    # Example: Remove all <script> and <style> tags
    for script_or_style in soup(["script", "style", "link", "svg"]):
        script_or_style.decompose()

    # Additional cleaning can be done here by removing other unwanted tags

    return str(soup)


def save_html_to_file(html, filename):
    with open(filename, "w", encoding="utf-8") as file:
        file.write(html)
    print(f"HTML content saved to {filename}")

def get_page_name(url):
    pattern = r"https?://(?:www\.)?([^./]+)\.[^/]+/([^/?#]*)"

    # Match the pattern against the URL
    match = re.search(pattern, url)

    if match:
        domain = match.group(1)
        page = match.group(2)
        
        # Clean the page part by removing any special characters after the username
        page_cleaned = re.split(r'[^a-zA-Z0-9]', page)[0]
        
        result = f"{domain}_{page_cleaned}"
        return result

In [164]:
from backend.fetcher.fetcher import fetch_page

url = "https://www.nguoiduatin.vn/clip-voi-vang-2-xe-may-va-cham-cuc-manh-2-tai-xe-vang-moi-nguoi-mot-noi-a667066.html"
html = await fetch_page(
    url,
    static_fetch=True,
    # expand_button_text=[
    #     "View all",
    #     "See more",
    #     "View more comments, View all (\d+) comments",
    #     "Xem tất cả (\d+) phản hồi",
    #     "Xem (\d+) phản hồi",
    #     "Xem thêm",
    # ],
)

In [ ]:
url = "https://vnexpress.net/bo-truong-tu-phap-le-thanh-long-lam-pho-thu-tuong-4751867.html"
vnexpress_html = await fetch_page(
    url,
    static_fetch=True
)
vnexpress_soup = prepare_html(vnexpress_html)

In [165]:
html

'<!DOCTYPE html>\n<html lang="vi">\n<head> <title>Clip: Vội vàng, 2 xe máy va chạm cực mạnh, 2 tài xế văng mỗi người một nơi</title> <meta http-equiv="content-language" content="vi" /> <meta name="language" content="vietnamese" /> <meta name="format-detection" content="telephone=no"> <link rel="shortcut icon" type="image/x-icon" href="/images/favicon.ico" /> <meta name="viewport" content="width=device-width, initial-scale=1"> <meta name="copyright" content="Copyright © 2017 by NguoiDuaTin.vn" /> <meta property="fb:pages" content="211599842221985"/> <meta property="fb:app_id" content="112216465254411"/>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/> <meta itemprop="description" name="description" content="Nữ tài xế vừa rẽ sang đường thì một xe máy do nam tài xế điều khiển lao tới tông trúng đầu xe. Va chạm cực mạnh khiến nữ tài xế văng xuống đường, còn nam tài xế văng về phía trước hàng chục mét."/> <link rel="canonical" href="https://www.nguoiduatin.vn/clip-voi-v

# Container Extract

In [150]:
example_container = {
    "article": "Hình ảnh vụ tai nạn được chia sẻ trên nhóm Facebook về giao thông mới đây cho thấy, 2 tài xế sau khi bị hất văng xuống đường sau một hồi nằm dưới đường đã có thể đứng dậy di chuyển vào lề đường.",
}

In [151]:


extractor = ExtractorTask(html, example_container)

2024-06-06 02:56:23,141 - utils.py: parse_html(): 79:	Parsing HTML string to BeautifulSoup object
--------
2024-06-06 02:56:23,172 - utils.py: clean_html(): 16:	Cleaning HTML, removing all path that does not contains navigable string
--------


[document] -> html -> head -> title -> Clip: Vội vàng, 2 xe máy va chạm cực mạnh, 2 tài xế văng mỗi người một nơi
[document] -> html -> body -> header -> section -> div -> div -> div -> div -> div -> div -> a -> Liên hệ
[document] -> html -> body -> header -> section -> div -> div -> div -> div -> div -> div -> a -> Góp ý
[document] -> html -> body -> header -> section -> div -> div -> div -> div -> div -> div -> a -> Quảng cáo
[document] -> html -> body -> header -> section -> div -> div -> ul -> li -> a -> Toàn cảnh
[document] -> html -> body -> header -> section -> div -> div -> ul -> li -> a -> Pháp luật
[document] -> html -> body -> header -> section -> div -> div -> ul -> li -> a -> Kinh tế
[document] -> html -> body -> header -> section -> div -> div -> ul -> li -> a -> Xã hội
[document] -> html -> body -> header -> section -> div -> div -> ul -> li -> a -> Đời sống
[document] -> html -> body -> header -> section -> div -> div -> ul -> li -> a -> Đa chiều
[document] -> html -> b

In [153]:
extractor.process_container

{'article': {'content': 'Hình ảnh vụ tai nạn được chia sẻ trên nhóm Facebook về giao thông mới đây cho thấy, 2 tài xế sau khi bị hất văng xuống đường sau một hồi nằm dưới đường đã có thể đứng dậy di chuyển vào lề đường.'}}

In [152]:
structured_extracted_contents, posts = await extractor.run_extractor_task()
# Modify container extract run -> 

2024-06-06 02:56:34,163 - container_extractor.py: prepare_extract(): 41:	Preparing extract
--------


Exception: Cannot find tag

In [135]:
import json


json.dumps(structured_extracted_contents, ensure_ascii=False)

'[{"title": ["Tổng thống Mỹ đến Pháp dự kỷ niệm 80 năm ngày D-Day"], "article": ["Trong hai ngày 6 - 7/6, Tổng thống Biden sẽ tham dự lễ kỷ niệm ngày D-Day tại vùng Normandy, phía Tây Bắc nước Pháp và có bài phát biểu quan trọng tại sự kiện. Sau đó, Tổng thống Mỹ sẽ bắt đầu chuyến thăm cấp nhà nước tại quốc gia châu Âu này.", "Cố vấn An ninh quốc gia Mỹ Jake Sullivan cho biết bài phát biểu của Tổng thống Biden ở Normandy sẽ đề cập tới sự nguy hiểm của chủ nghĩa biệt lập. Ông Biden sẽ rút ra mối liên hệ từ hai cuộc Chiến tranh Thế giới thứ hai và Chiến tranh Lạnh, với việc thành lập Tổ chức Hiệp ước Bắc Đại Tây Dương (NATO) cho đến ngày nay.", "Đặc biệt, Lễ kỷ niệm D-Day được đặt trong bối cảnh xung đột đang trở lại châu Âu và các nước NATO đang gia tăng sức mạnh quân sự. Tổng thống Biden cũng sẽ gặp lại những cựu chiến binh lớn tuổi đã tham gia D-Day.", "Về thông tin cho rằng Tổng thống Pháp Emmanuel Macron đang cân nhắc việc cử cố vấn quân sự đến Ukraine, điều phối viên về truyền thôn

In [126]:
structured_extracted_contents

[{'title': ['Tổng thống Mỹ đến Pháp dự kỷ niệm 80 năm ngày D-Day'],
  'article': ['Trong hai ngày 6 - 7/6, Tổng thống Biden sẽ tham dự lễ kỷ niệm ngày D-Day tại vùng Normandy, phía Tây Bắc nước Pháp và có bài phát biểu quan trọng tại sự kiện. Sau đó, Tổng thống Mỹ sẽ bắt đầu chuyến thăm cấp nhà nước tại quốc gia châu Âu này.',
   'Cố vấn An ninh quốc gia Mỹ Jake Sullivan cho biết bài phát biểu của Tổng thống Biden ở Normandy sẽ đề cập tới sự nguy hiểm của chủ nghĩa biệt lập. Ông Biden sẽ rút ra mối liên hệ từ hai cuộc Chiến tranh Thế giới thứ hai và Chiến tranh Lạnh, với việc thành lập Tổ chức Hiệp ước Bắc Đại Tây Dương (NATO) cho đến ngày nay.',
   'Đặc biệt, Lễ kỷ niệm D-Day được đặt trong bối cảnh xung đột đang trở lại châu Âu và các nước NATO đang gia tăng sức mạnh quân sự. Tổng thống Biden cũng sẽ gặp lại những cựu chiến binh lớn tuổi đã tham gia D-Day.',
   'Về thông tin cho rằng Tổng thống Pháp Emmanuel Macron đang cân nhắc việc cử cố vấn quân sự đến Ukraine, điều phối viên về t

In [ ]:
for i, post in enumerate(structured_extracted_contents):
    if (post["tag"] == "IU Confessions") or (
        not post["content"] and not post["content2"]
    ):
        structured_extracted_contents.pop(i)

In [48]:
import re


pattern = re.compile(r"(Expand|See more|Xem thêm|Hiển thị|View all (\d+) comments)")
text = "View all 50 comments"

if re.search(pattern, text):
    print("Match found")
else:
    print("No match")

Match found


In [46]:
button_text = ["View all", "See more", "HI"]

print("|".join(button_text))

View all|See more|HI


In [5]:
import re
remove_line = re.sub(r"(\r\n|\r|\n){2,}", lambda x: "\n", cleaned_html_with_see_more)

In [9]:
cleaned_html_with_see_more

'<!DOCTYPE html>\n<html class="_9dls __fb-light-mode" dir="ltr" id="facebook" lang="vi"><head><meta content="noarchive" name="bingbot"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs..." name="description"/><meta content="video.other" property="og:type"/><meta content="IU Confessions" property="og:title"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs và nội dung các bài đó đa phần đánh giá trường..." property="og:description"/><meta content="https://www.facebook.com/HcmIUcfs/posts/iu_cfs11567-31052024-95513em-ch%C3%A0o-c%C3%A1c-anh-ch%E1%BB%8B-%E1%BA%A1-chuy%E1%BB%87n-l%C3%A0-em-s%E1%BA%AFp-thi-tnthpt-v%C3%A0-c/894616432709339/" property="og:url"

In [8]:
remove_line

'<!DOCTYPE html>\n<html class="_9dls __fb-light-mode" dir="ltr" id="facebook" lang="vi"><head><meta content="noarchive" name="bingbot"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs..." name="description"/><meta content="video.other" property="og:type"/><meta content="IU Confessions" property="og:title"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs và nội dung các bài đó đa phần đánh giá trường..." property="og:description"/><meta content="https://www.facebook.com/HcmIUcfs/posts/iu_cfs11567-31052024-95513em-ch%C3%A0o-c%C3%A1c-anh-ch%E1%BB%8B-%E1%BA%A1-chuy%E1%BB%87n-l%C3%A0-em-s%E1%BA%AFp-thi-tnthpt-v%C3%A0-c/894616432709339/" property="og:url"

In [10]:
expand_text = "Show more, See more, Expand"
expand_button_text = expand_text.split(", ")
expand_button_text

['Show more', 'See more', 'Expand']

# Singular Extract

In [234]:
from backend.extractor.task.single_element_extractor import SingleElementExtractor


example_container = {
    "article": [
        "Bộ trưởng Tư pháp Lê Thành Long được Quốc hội phê chuẩn bổ nhiệm làm Phó thủ tướng nhiệm kỳ 2021-2026, chiều 6/6.",
        """Nghị quyết phê chuẩn đề nghị của Thủ tướng về việc bổ nhiệm Phó thủ tướng nhiệm kỳ 2021-2026 với ông Lê Thành Long thực hiện bằng hình thức biểu quyết điện tử. 468/469 đại biểu tham gia biểu quyết đã bấm nút "tán thành"""
    ]
}
single_extractor = SingleElementExtractor(example_container, vnexpress_html)

In [184]:
single_extractor.process_container

{'article': {'content': ['Nữ tài xế vừa rẽ sang đường thì một xe máy do nam tài xế điều khiển lao tới tông trúng đầu xe. Va chạm cực mạnh khiến nữ tài xế văng xuống đường, còn nam tài xế văng về phía trước hàng chục mét.',
   'Hình ảnh vụ tai nạn được chia sẻ trên nhóm Facebook về giao thông mới đây cho thấy, 2 tài xế sau khi bị hất văng xuống đường sau một hồi nằm dưới đường đã có thể đứng dậy di chuyển vào lề đường.']}}

In [ ]:
soup = prepare_html(html)

In [224]:
from backend.extractor.extractor import find_string_tag


tag = find_string_tag(
    soup=soup,
    navigable_string_content="Nữ tài xế vừa rẽ sang đường thì một xe máy do nam tài xế điều khiển lao tới tông trúng đầu xe. Va chạm cực mạnh khiến nữ tài xế văng xuống đường, còn nam tài xế văng về phía trước hàng chục mét.",
)
tag.attrs

2024-06-06 17:24:20,335 - extractor.py: find_string_tag(): 16:	Found string in soup at: <div class="tmp-title-large my-4"> Ná»¯ tÃ i xáº¿ vá»«a ráº½ sang ÄÆ°á»ng thÃ¬ má»t xe mÃ¡y do nam tÃ i xáº¿ Äiá»u khiá»n lao tá»i tÃ´ng trÃºng Äáº§u xe. Va cháº¡m cá»±c máº¡nh khiáº¿n ná»¯ tÃ i xáº¿ vÄng xuá»ng ÄÆ°á»ng, cÃ²n nam tÃ i xáº¿ vÄng vá» phÃ­a trÆ°á»c hÃ ng chá»¥c mÃ©t. </div>
--------


{'class': ['tmp-title-large', 'my-4']}

In [ ]:
def narrow_search(soup, content):
    target_tag = find_string_tag(soup, content)
    attributes = target_tag.attrs
    if "id" in attributes.keys():
        tag_id = "#" + attributes["id"]
    if "class" in attributes.keys():
        tag_class = ".".join(attributes["class"])
    
    

In [219]:
res = []
path = f"html body section div div div div div.tmp-title-large.my-4"
selector = path.replace(" ", " > ")

similar_tags = soup.select(selector)
string_values = (
    [tag.get_text().strip() for tag in similar_tags]
    if similar_tags is not None
    else ""
)
res.append(string_values)
res

[['Nữ tài xế vừa rẽ sang đường thì một xe máy do nam tài xế điều khiển lao tới tông trúng đầu xe. Va chạm cực mạnh khiến nữ tài xế văng xuống đường, còn nam tài xế văng về phía trước hàng chục mét.']]

In [235]:
single_extractor.extract_similar_text_from_example(vnexpress_soup, single_extractor.process_container["article"]["content"])

2024-06-06 17:34:48,136 - single_element_extractor.py: extract_similar_text_from_example(): 30:	Getting tags
--------
2024-06-06 17:34:48,144 - extractor.py: find_string_tag(): 16:	Found string in soup at: <p class="description">Bá» trÆ°á»ng TÆ° phÃ¡p LÃª ThÃ nh Long ÄÆ°á»£c Quá»c há»i phÃª chuáº©n bá» nhiá»m lÃ m PhÃ³ thá»§ tÆ°á»ng nhiá»m ká»³ 2021-2026, chiá»u 6/6.</p>
--------
2024-06-06 17:34:48,163 - extractor.py: find_string_tag(): 16:	Found string in soup at: <p class="Normal">Nghá» quyáº¿t phÃª chuáº©n Äá» nghá» cá»§a Thá»§ tÆ°á»ng vá» viá»c bá» nhiá»m PhÃ³ thá»§ tÆ°á»ng nhiá»m ká»³ 2021-2026 vá»i Ã´ng LÃª ThÃ nh Long thá»±c hiá»n báº±ng hÃ¬nh thá»©c biá»u quyáº¿t Äiá»n tá»­. 468/469 Äáº¡i biá»u tham gia biá»u quyáº¿t ÄÃ£ báº¥m nÃºt "tÃ¡n thÃ nh".</p>
--------
2024-06-06 17:34:48,165 - extractor.py: find_path(): 69:	Extract path name: html body section div div p
--------
2024-06-06 17:34:48,166 - extractor.py: find_path(): 69:	Extract path name: ht

<p class="description">Bộ trưởng Tư pháp Lê Thành Long được Quốc hội phê chuẩn bổ nhiệm làm Phó thủ tướng nhiệm kỳ 2021-2026, chiều 6/6.</p>
<p class="Normal">Nghị quyết phê chuẩn đề nghị của Thủ tướng về việc bổ nhiệm Phó thủ tướng nhiệm kỳ 2021-2026 với ông Lê Thành Long thực hiện bằng hình thức biểu quyết điện tử. 468/469 đại biểu tham gia biểu quyết đã bấm nút "tán thành".</p>
['html body section div div p', 'html body section div div article p']


[['Bộ trưởng Tư pháp Lê Thành Long được Quốc hội phê chuẩn bổ nhiệm làm Phó thủ tướng nhiệm kỳ 2021-2026, chiều 6/6.'],
 ['Nghị quyết phê chuẩn đề nghị của Thủ tướng về việc bổ nhiệm Phó thủ tướng nhiệm kỳ 2021-2026 với ông Lê Thành Long thực hiện bằng hình thức biểu quyết điện tử. 468/469 đại biểu tham gia biểu quyết đã bấm nút "tán thành".',
  'Ông Lê Thành Long 61 tuổi, quê thành phố Sầm Sơn, tỉnh Thanh Hóa, Tiến sĩ Luật. Ông là Ủy viên Trung ương Đảng khóa 12, 13; Đại biểu Quốc hội khóa 14, 15.',
  'Sau khi tốt nghiệp Cử nhân Luật tại Đại học tổng hợp Adecbaijan (Liên Xô cũ), ông Long bắt đầu sự nghiệp ở vị trí chuyên viên Vụ Hợp tác quốc tế, Bộ Tư pháp. Ông sau đó trải qua nhiều cương vị khác nhau ở Bộ Tư pháp như thư ký Bộ trưởng, Vụ trưởng Các vấn đề chung về xây dựng pháp luật, Thứ trưởng Tư pháp.',
  'Trong hơn một năm từ tháng 3/2014 đến 10/2015, ông Long làm Phó bí thư Tỉnh ủy Hà Tĩnh. Từ tháng 4/2016 đến nay, ông là Bí thư Ban cán sự Đảng, Bộ trưởng Tư pháp.',
  'Chính phủ 

In [105]:
article_contents = []
for path in paths:
    selector = path.replace(" ", " > ")
    similar_tags = cleaned_html.select(selector)
    string_values = (
        [tag.get_text().strip() for tag in similar_tags]
        if similar_tags is not None
        else ""
    )
    article_contents.append(string_values)

In [106]:
article_contents

[['Nghi phạm - quốc tịch Syria - được đưa đến bệnh viện cấp cứu và các binh sĩ đang kiểm tra hiện trường.',
  'Đại sứ quán Mỹ xác nhận thông tin có một vụ nổ súng gần lối vào. Tòa nhà và các nhân viên vẫn an toàn. Theo các nguồn tin ngoại giao, Đại sứ Mỹ Lisa Johnson đang công du nước ngoài.',
  'Một nguồn tin nói với Reuters rằng một nhân viên an ninh tại đại sứ quán đã bị thương trong vụ tấn công, và nghi phạm cũng bị thương ở bụng.',
  'Reuters đã xác minh hình ảnh của nghi phạm lan truyền trên mạng. Trên áo của nghi phạm có dòng chữ "Hồi giáo".',
  'Quyền Thủ tướng Li-băng Najib Mikati cho biết ông đang theo dõi vụ việc với bộ trưởng quốc phòng và lực lượng an ninh.',
  'Đại sứ quán Mỹ nằm ở Awkar, phía bắc Beirut trong một khu vực được bảo vệ nghiêm ngặt với nhiều trạm kiểm soát dọc theo lối vào. Đại sứ quán chuyển đến đó từ Beirut sau vụ tấn công tự sát năm 1983 khiến hơn 60 người thiệt mạng.',
  'Hồi tháng 9, có tiếng súng nổ gần đại sứ quán nhưng không có báo cáo về thương tích

In [25]:
import pandas as pd

dictionary = {}
df = pd.DataFrame({"col1": [1], "": [0.5]})

for col in df.columns:
    if col != "": 
        dictionary[col] = df[col].values.tolist()
# dictionary = dataframe.to_dict()

In [26]:
dictionary

{'col1': [1]}

In [11]:
df["col1"].values

array([1, 2])